- OpenClassrooms Project 7, Data Scientist
- Author : Oumeima EL GHARBI
- Date : November, December 2022

### Importing libraries

In [19]:
%reset -f

import warnings
warnings.filterwarnings(action="ignore")

# to compute time of pipeline
from time import time, strftime, gmtime

import pandas as pd
from os import listdir
from os.path import isfile, join

%matplotlib inline
%autosave 300

Autosaving every 300 seconds


In [20]:
import sys
print("Python version {}.{}.{}".format(str(sys.version_info[0]), str(sys.version_info[1]), str(sys.version_info[2])))

Python version 3.9.7


In [21]:
# Starting time
t0 = time()

input_path = "./dataset/source/"

output_path = "./dataset/cleaned/"

## Part I : Exploratory Data Analysis

#### Importing all datasets

We read the folder in which we have all the csv files.

In [22]:
all_files = [f for f in listdir(input_path) if isfile(join(input_path, f))]

all_csv_files = [file for file in all_files if file.endswith(".csv")]
print(all_files, end='\n\n')

print(all_csv_files)

['application_test.csv', 'application_train.csv', 'bureau.csv', 'bureau_balance.csv', 'credit_card_balance.csv', 'HomeCredit_columns_description.csv', 'installments_payments.csv', 'POS_CASH_balance.csv', 'previous_application.csv', 'sample_submission.csv']

['application_test.csv', 'application_train.csv', 'bureau.csv', 'bureau_balance.csv', 'credit_card_balance.csv', 'HomeCredit_columns_description.csv', 'installments_payments.csv', 'POS_CASH_balance.csv', 'previous_application.csv', 'sample_submission.csv']


In [23]:
def load_data(path, filename):
    """
    Step 0)
    :param path:
    :param filename: (string)
    :return:
    """
    print("___Loading raw dataset___")

    # Load raw data
    dataset_file = "{}{}".format(path, filename)
    dataset = pd.read_csv(dataset_file, encoding="windows-1252")

    print("Initial shape :", dataset.shape)
    return dataset

In [24]:
all_datasets = []

for index, dataset_csv in enumerate(all_csv_files):
    name = dataset_csv.split(".")[0]
    if name.startswith("olist"):
        list_name = name.split("_")[1:-1]
        name = "_".join(list_name)
    # we make the dataset as a global variable so we can use it later in the notebook
    globals()["dataset_" + name] = load_data(input_path, dataset_csv)

    # we add the dataset to the list of all datasets.
    #all_datasets.append(globals()["dataset_" + name])

    print("This is the dataset n°{} : {}".format(index + 1, name))
    df = globals()["dataset_" + name]
    all_datasets.append(("dataset_" + name, df))
    display(df.head(2))
    display(df.info(verbose=True))

___Loading raw dataset___
Initial shape : (48744, 121)
This is the dataset n°1 : application_test


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48744 entries, 0 to 48743
Data columns (total 121 columns):
 #    Column                        Dtype  
---   ------                        -----  
 0    SK_ID_CURR                    int64  
 1    NAME_CONTRACT_TYPE            object 
 2    CODE_GENDER                   object 
 3    FLAG_OWN_CAR                  object 
 4    FLAG_OWN_REALTY               object 
 5    CNT_CHILDREN                  int64  
 6    AMT_INCOME_TOTAL              float64
 7    AMT_CREDIT                    float64
 8    AMT_ANNUITY                   float64
 9    AMT_GOODS_PRICE               float64
 10   NAME_TYPE_SUITE               object 
 11   NAME_INCOME_TYPE              object 
 12   NAME_EDUCATION_TYPE           object 
 13   NAME_FAMILY_STATUS            object 
 14   NAME_HOUSING_TYPE             object 
 15   REGION_POPULATION_RELATIVE    float64
 16   DAYS_BIRTH                    int64  
 17   DAYS_EMPLOYED                 int64  
 18   DAYS

None

___Loading raw dataset___
Initial shape : (307511, 122)
This is the dataset n°2 : application_train


...

None

___Loading raw dataset___
Initial shape : (48744, 2)
This is the dataset n°10 : sample_submission


,SK_ID_CURR,TARGET
0,100001,0.5
1,100005,0.5


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48744 entries, 0 to 48743
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SK_ID_CURR  48744 non-null  int64  
 1   TARGET      48744 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 761.8 KB


None

In [25]:
test = pd.read_csv(input_path + "HomeCredit_columns_description.csv", encoding="windows-1252")
test

,Unnamed: 0,Table,Row,Description,Special
0,1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
1,2,application_{train|test}.csv,TARGET,Target variable (1 - client with payment diffi...,NaN
2,5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
3,6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
4,7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN
...,...,...,...,...,...
214,217,installments_payments.csv,NUM_INSTALMENT_NUMBER,On which installment we observe payment,NaN
215,218,installments_payments.csv,DAYS_INSTALMENT,When the installment of previous credit was su...,time only relative to the application
216,219,installments_payments.csv,DAYS_ENTRY_PAYMENT,When was the installments of previous credit p...,time only relative to the application
217,220,installments_payments.csv,AMT_INSTALMENT,What was the prescribed installment amount of ...,NaN


For each dataset, we display its dimension and if it has missing values (NaN)

In [26]:
print("Shape dataset")
for tuple_df in all_datasets :
    print("The dataset called :", tuple_df[0], "has a shape :", tuple_df[1].shape)

Shape dataset
The dataset called : dataset_application_test has a shape : (48744, 121)
The dataset called : dataset_application_train has a shape : (307511, 122)
The dataset called : dataset_bureau has a shape : (1716428, 17)
The dataset called : dataset_bureau_balance has a shape : (27299925, 3)
The dataset called : dataset_credit_card_balance has a shape : (3840312, 23)
The dataset called : dataset_HomeCredit_columns_description has a shape : (219, 5)
The dataset called : dataset_installments_payments has a shape : (13605401, 8)
The dataset called : dataset_POS_CASH_balance has a shape : (10001358, 8)
The dataset called : dataset_previous_application has a shape : (1670214, 37)
The dataset called : dataset_sample_submission has a shape : (48744, 2)


In [27]:
print("Missing values")
for tuple_df in all_datasets :
    df = tuple_df[1]
    print("The dataset called :", tuple_df[0], "has :", df.isna().sum().sum() / (df.shape[0] * df.shape[1]) * 100, "% of missing values.")

Missing values
The dataset called : dataset_application_test has : 23.811686761532336 % of missing values.
The dataset called : dataset_application_train has : 24.39594190712943 % of missing values.
The dataset called : dataset_bureau has : 13.502552290879322 % of missing values.
The dataset called : dataset_bureau_balance has : 0.0 % of missing values.
The dataset called : dataset_credit_card_balance has : 6.654074392687479 % of missing values.
The dataset called : dataset_HomeCredit_columns_description has : 12.146118721461187 % of missing values.
The dataset called : dataset_installments_payments has : 0.005337953655316738 % of missing values.
The dataset called : dataset_POS_CASH_balance has : 0.06518864738168557 % of missing values.
The dataset called : dataset_previous_application has : 17.97687747344498 % of missing values.
The dataset called : dataset_sample_submission has : 0.0 % of missing values.


In [28]:
dataset_application_test.isna().sum(axis=0)

SK_ID_CURR                       0
NAME_CONTRACT_TYPE               0
CODE_GENDER                      0
FLAG_OWN_CAR                     0
FLAG_OWN_REALTY                  0
                              ... 
AMT_REQ_CREDIT_BUREAU_DAY     6049
AMT_REQ_CREDIT_BUREAU_WEEK    6049
AMT_REQ_CREDIT_BUREAU_MON     6049
AMT_REQ_CREDIT_BUREAU_QRT     6049
AMT_REQ_CREDIT_BUREAU_YEAR    6049
Length: 121, dtype: int64

In [29]:
print("Duplicated rows ?")
for tuple_df in all_datasets:
    print("The dataset called :", tuple_df[0], "has :",tuple_df[1].duplicated().sum(), "duplicated rows.")

Duplicated rows ?
The dataset called : dataset_application_test has : 0 duplicated rows.
The dataset called : dataset_application_train has : 0 duplicated rows.
The dataset called : dataset_bureau has : 0 duplicated rows.
The dataset called : dataset_bureau_balance has : 0 duplicated rows.
The dataset called : dataset_credit_card_balance has : 0 duplicated rows.
The dataset called : dataset_HomeCredit_columns_description has : 0 duplicated rows.
The dataset called : dataset_installments_payments has : 0 duplicated rows.
The dataset called : dataset_POS_CASH_balance has : 0 duplicated rows.
The dataset called : dataset_previous_application has : 0 duplicated rows.
The dataset called : dataset_sample_submission has : 0 duplicated rows.


In [30]:
# End of notebook time
t1 = time()
print("computing time : {:8.6f} sec".format(t1 - t0))
print("computing time : " + strftime('%H:%M:%S', gmtime(t1 - t0)))

computing time : 119.102574 sec
computing time : 00:01:59
